In [1]:
%load_ext autoreload
%autoreload 2

from fastai.text.all import *
import pandas as pd
import torch
from tqdm.notebook import tqdm

In [2]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

# tensorflow RNG
# tf.random.set_seed(seed)

In [3]:
data_path = Path("/mnt/nas/backups/08-07-2020/desktopg01/lisa/Data/CSV")

In [4]:
model_path=Path("./models")

In [5]:
train_csv = pd.read_csv(data_path/"train_small.csv", usecols=['document_type', 'body'])
val_csv = pd.read_csv(data_path/"validation_small.csv", usecols=['document_type', 'body'])
test_csv = pd.read_csv(data_path/"test_small.csv", usecols=['document_type', 'body'])

In [6]:
train_csv["is_valid"] = False
val_csv["is_valid"] = True
data = pd.concat([train_csv, val_csv]); data.head()

,document_type,body,is_valid
0,outros,"{""tribunal justiça estado bahia poder judiciário salvador juizado cível defesa consumidor universo matutino projudi acm campus universidade salgado oliveira universo pituba salvador processo parte autora nailton lantyer cordeiro araujo parte atlantico fundo investimento direitos creditorios despacho vistos etc indefiro requerido pela parte demandante aguarde a sessão conciliação designada salvador de julho paulo alberto nunes chenaud juiz direito documento assinado eletronicamente""}",False
1,outros,"{""excelentíssimo senhor doutor juiz direito juizado cível defesa consumidor universo comarca salvador autos processo º atlântico fundo investimento direitos creditórios não padronizados nova denominação crdg fundo investimento direitos creditórios não padronizados empresa inscrita cnpj sob º com sede av brigadeiro faria lima andar são paulo por seus advogados signatários nos autos ação epígrafe proposta por nailton lantyer cordeiro araujo vem respeitosamente presença vossa excelência inconformado com r sentença fls com fulcro ARTIGO_41 lei º interpor presente recurso inominado ser recebido...",False
2,outros,"{""razões recurso inominado recorrente atlantico fundo investimento direitos creditórios não padronizados recorrido nailton lantyer cordeiro araujo autos processo º origem jec comarca salvador colendo colégio recursal ilustres julgadores síntese demanda parte recorrida intentou presente demanda alegando síntese que teve seu nome indevidamente incluído cadastro proteção crédito que diligenciar sentido obter informações acerca inscrição verificou que negativação foi ordenada pelo recorrente sendo assim propôs presente demanda pleiteando exclusão seus dados spc serasa inexigibilidade débito o ...",False
3,outros,"{""empresa recorrente tornou credora dos débitos nome parte recorrida referente uma linha telefônica razão contrato cessão crédito entre empresa originalmente credora cedente o atlântico fundo investimentos direitos creditórios não padronizados ora cessionária com efeito caso não houve qualquer cobrança indevida afinal tornando a parte recorrida inadimplente vindo parte recorrente firmar contrato cessão direitos creditórios junto empresa credora recorrido totalmente legítima cobrança perpetrada cumpre esclarecer que época cessão parte recorrida estava inadimplente com empresa recorrente mot...",False
4,outros,"{""entretanto verdade parte apelante tornou titular crédito devido pela parte apelada por força um contrato cessão crédito celebrado com brasil telecom a seja que pese parte apelada não ter firmado inicialmente contrato diretamente junto apelante este tornou seu credor virtude aludido instrumento cessão crédito portanto caso seja comprovada alegação perpetrada pela parte apelada que o contrato que originou seu débito seja fraudulento quem deverá arcar com eventuais ônus a brasil telecom a não esta parte apelante que não foi quem contratou com parte apelada por sua vez parte apelada foi regu...",False


In [7]:
tok = SentencePieceTokenizer(sp_model=model_path/"spm.model")

In [ ]:
dls_lm = DataBlock(
    blocks=(TextBlock.from_df('body', tok=tok, is_lm=True)),
    get_x=ColReader('text'), splitter=ColSplitter()).dataloaders(data)

In [ ]:
torch.save(dls_lm, "./data/dls_lm.pth")

In [ ]:
# dls_lm = torch.load("./data/dls_lm.pth")

In [ ]:
assert len(train_csv) == len(dls_lm.train_ds) and len(val_csv) ==len(dls_lm.valid_ds)

In [ ]:
dls_lm.show_batch()

In [ ]:
config = awd_lstm_lm_config.copy()
config['qrnn'] = True
config['n_hid'] = 1550 #default 1152
config['n_layers'] = 4 #default 3

In [ ]:
perplexity = Perplexity()
f1 = F1Score(average='macro')

In [ ]:
lm_fns3 = ['pt_wt_sp15_multifit', 'pt_wt_vocab_sp15_multifit']
lm_fns3_bwd = ['pt_wt_sp15_multifit_bwd', 'pt_wt_vocab_sp15_multifit_bwd']

In [ ]:
learn_lm = language_model_learner(dls_lm, AWD_LSTM, path=Path("."), config=config, pretrained_fnames=lm_fns3, drop_mult=1., 
                                  metrics=[error_rate, accuracy, perplexity])

In [ ]:
learn_lm.save_encoder("no_fine_tune_enc")

In [ ]:
TEXT = "O Ministro"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn_lm.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
learn_lm.freeze()
learn_lm.summary()

In [ ]:
learn_lm.lr_find()